# Rapid API post

- title: Rapid API post
- toc: true 
- badges: true
- comments: true
- categories: [jupyter]

In [3]:
HttpRequest request = HttpRequest.newBuilder()
		.uri(URI.create("https://airbnb19.p.rapidapi.com/api/v1/getLanguages"))
		.header("X-RapidAPI-Key", "3bef89a477msh7eecb5e7498cd51p1095f2jsn9b36f511d7ec")
		.header("X-RapidAPI-Host", "airbnb19.p.rapidapi.com")
		.method("GET", HttpRequest.BodyPublishers.noBody())
		.build();
HttpResponse<String> response = HttpClient.newHttpClient().send(request, HttpResponse.BodyHandlers.ofString());
System.out.println(response.body());

{"status":true,"message":"Success","timestamp":1664866535098,"data":[{"id":"ar-SA","title":"Arabic (Saudi Arabia)"},{"id":"zh-CN","title":"Chinese (China)"},{"id":"zh-HK","title":"Chinese (Hong Kong SAR China)"},{"id":"zh-TW","title":"Chinese (Taiwan)"},{"id":"cs-CZ","title":"Czech (Czech Republic)"},{"id":"da-DK","title":"Danish (Denmark)"},{"id":"nl-BE","title":"Dutch (Belgium)"},{"id":"nl-NL","title":"Dutch (Netherlands)"},{"id":"en-AU","title":"English (Australia)"},{"id":"en-IE","title":"English (Ireland)"},{"id":"en-ZA","title":"English (South Africa)"},{"id":"en-GB","title":"English (United Kingdom)"},{"id":"en-US","title":"English (United States)"},{"id":"fi-FI","title":"Finnish (Finland)"},{"id":"fr-CA","title":"French (Canada)"},{"id":"fr-FR","title":"French (France)"},{"id":"de-DE","title":"German (Germany)"},{"id":"el-GR","title":"Greek (Greece)"},{"id":"he-IL","title":"Hebrew (Israel)"},{"id":"hi-IN","title":"Hindi (India)"},{"id":"hu-HU","title":"Hungarian (Hungary)"},{"i